In [1]:
import os

os.environ['PROJECT_DIR'] = '/staff/tord/Workspace/arces_classification'
os.environ['DATA_DIR'] = '/projects/Array/ML_DataSet'
from global_config import cfg
from src.Utils import *
import pandas as pd
import datetime
from h5py import File

2024-03-25 16:11:36.110495: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-25 16:11:36.165404: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 16:11:38.087753: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:

beams_noise, beams, labels_event_type, metadata = get_file_names(cfg)
all_events = event_mapper(beams_noise, labels_event_type, metadata, cfg)

1998
1999
2000
2022
2010
2001
2002
2003
2004
2005
2006
2007
2008
2009
2011
2012
2013
2014
2015


In [5]:
import h5py

def print_structure(name, obj):
    print(name)
    if isinstance(obj, h5py.Dataset):
        print('Dataset, shape:', obj.shape, ', dtype:', obj.dtype)
        if obj.shape[0] > 0:  # If the dataset has entries
            print('First entry:', obj[0])  # Print the first entry

def explore_hdf5(file_path):
    with h5py.File(file_path, 'r') as f:
        f.visititems(print_structure)

In [7]:
#explore_hdf5(labels_event_type[0])
explore_hdf5(beams[0])
#explore_hdf5(beams_noise[0])
print(pd.read_csv(get_file_by_year(metadata, 1998)).head())

X
Dataset, shape: (296, 21600, 3) , dtype: float32
First entry: [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 4.17458459e-08  2.45993510e-06  5.09474512e-06]
 [ 7.49072797e-07  1.12670634e-04  2.76310719e-04]
 ...
 [-2.51419568e+00 -4.50670671e+00  1.36685860e+00]
 [ 5.67417431e+00 -4.94944286e+00  5.11830521e+00]
 [ 9.10045624e+00 -4.51833487e+00  6.79492903e+00]]
arrivals
Dataset, shape: (296,) , dtype: object
First entry: b'event_1998_90_0,event_1998_90_1,event_1998_90_2,event_1998_90_3'
event_id
Dataset, shape: (296,) , dtype: object
First entry: b'event_1998_90'
station
Dataset, shape: (296,) , dtype: object
First entry: b'ARA0'
window
Dataset, shape: (296, 2) , dtype: float64
First entry: [8.85096452e+08 8.85096992e+08]
   Unnamed: 0  arrival_id                         time label station  snr  \
0           0           0  1998-01-01T22:41:44.297000Z     P   ARCES  6.3   
1           1           1  1998-01-01T22:43:17.500000Z     P   FINES  6.3   
2           2           0

In [7]:
beams_noise[2]

'/projects/Array/ML_DataSet/data/eventclass_2000_beams_noise.hdf5'

In [8]:
def event_mapper(noise_beams, labels_event_types, metadata, cfg):            
    def get_snr(metadata_df, event_id):
        snr = metadata_df[metadata_df['event_id'] == event_id]['snr']
        if snr.empty:
            return None
        if len(snr) > 1:
            # Use the highest snr:
            return snr.max()
        return snr
    events = []
    for i in range(len(labels_event_types)):
        year = int(labels_event_types[i].split("/")[-1].split("_")[1])
        print(year)
        with File(labels_event_types[i], 'r') as f:
            labels = f['labels'][:]
            ids = f['event_id'][:]
        metadata_df = pd.read_csv(get_file_by_year(metadata, year))
        for event_index, (label, event_id) in enumerate(zip(labels, ids)):
            snr = get_snr(metadata_df, event_id.decode('utf-8'))
            label = label.decode('utf-8')   
            if cfg.data.include_induced and "induced" in label:
                label = "earthquake"
            if not cfg.data.include_induced and "induced" in label:
                continue
            events.append((year, event_id.decode('utf-8'), label, snr, event_index))
        with File(get_file_by_year(noise_beams, year), 'r') as f:
            noise = f['station'][:]
            for noise_index in range(len(noise)):
                events.append((year, "not_applicable", "noise", None, noise_index))  # No SNR data for noise events
    return events

def get_file_by_year(files, year):
    for file in files:
        if str(year) in file.split("/")[-1]:
            #print(file)
            return file



def train_val_test_split(all_events, cfg):
    val_years = cfg.data.val_years
    test_years = cfg.data.test_years
    train_events, val_events, test_events = [], [], []
    for event in all_events:
        if event[0] in val_years:
            val_events.append(event)
        elif event[0] in test_years:
            test_events.append(event)
        else:
            train_events.append(event)
    return train_events, val_events, test_events

def get_file_names(cfg):
    beams_noise, beams, labels_event_type, metadata = [], [], [], []
    file_endings = ["_beams.hdf5", "_beams_noise.hdf5", "_labels_event_type.hdf5"]
    file_endings_metadata = ["snrupdate.csv"]
    for file in os.listdir(cfg.data_paths.data_path):
        path = cfg.data_paths.data_path + file
        if file.endswith(tuple(file_endings)):
            if file.endswith("_beams.hdf5"):
                beams.append(path)
            elif file.endswith("_beams_noise.hdf5"):
                beams_noise.append(path)
            elif file.endswith("_labels_event_type.hdf5"):
                labels_event_type.append(path)
    for file in os.listdir(cfg.data_paths.metadata_folder):
        path = cfg.data_paths.metadata_folder + file
        if file.endswith(tuple(file_endings_metadata)):
            metadata.append(path)
    return beams_noise, beams, labels_event_type, metadata

def calculate_class_weights(train_events, label_dict):
    class_counts = {label: 0 for label in label_dict.keys()}
    for event in train_events:
        class_counts[event[2]] += 1
    total = sum(class_counts.values())
    class_weights = {label: total / count for label, count in class_counts.items()}
    return class_weights, class_counts

def preprocessing_pipeline(cfg):
    beams_noise, beams, labels_event_type, metadata = get_file_names(cfg)
    all_events = event_mapper(beams_noise, labels_event_type, metadata, cfg)
    train_events, val_events, test_events = train_val_test_split(all_events, cfg)
    all_labels = list(set([event[2] for event in all_events]))
    label_dict = {label: i for i, label in enumerate(all_labels)}
    class_weights, class_counts = calculate_class_weights(train_events, label_dict)
    logger.info(f"Class counts: {class_counts}")
    logger.info("Loading data")
    train_dict = load_data_dict(train_events, beams, beams_noise)
    val_dict = load_data_dict(val_events, beams, beams_noise)
    test_dict = load_data_dict(test_events, beams, beams_noise)
    return train_events, val_events, test_events, all_events, label_dict, class_weights, train_dict, val_dict, test_dict

def load_data_dict(events, beams, beams_noise):
    # Creates a 
    data_dict = {}
    events_by_year = {}
    for event in events:
        year = event[0]
        if year not in data_dict.keys():
            data_dict[year] = {"events": {}, 
                               "noise": {}}
            events_by_year[year] = []
        events_by_year[year].append(event)
        
    for year in data_dict.keys():
        beam_file = get_file_by_year(beams, year)
        noise_file = get_file_by_year(beams_noise, year)
        with File(beam_file, 'r') as f:
            X = f['X'][:]
            X = np.transpose(X, (0, 2, 1))
            event_id = f['event_id'][:]
            for event_id, beam in zip(event_id, X):
                data_dict[year]["events"][event_id.decode('utf-8')] = beam
        with File(noise_file, 'r') as f:
            noise = f['X'][:]
            for i, n in enumerate(noise):
                data_dict[year]["noise"][i] = n
    return data_dict 
        
        

In [9]:
run_id = datetime.datetime.now().strftime("%y%m%d_%H%M%S") if not cfg.run_id else cfg.run_id
cfg = prepare_folders_paths_cfg(run_id, cfg, make_folders = True)
train_events, val_events, test_events, all_events, label_dict, class_weights, train_dict, val_dict, test_dict = preprocessing_pipeline(cfg)

1998
1999
2000
2022
2010
2001
2002
2003
2004
2005
2006
2007
2008
2009
2011
2012
2013
2014
2015
ARCES - 3499241424.py:87 - preprocessing_pipeline - INFO: Class counts: {'explosion': 29940, 'noise': 41281, 'earthquake': 11963}
ARCES - 3499241424.py:88 - preprocessing_pipeline - INFO: Loading data


In [12]:
example = train_dict[1998]["events"]['event_1998_90']

In [10]:
class RandomCropTransfrom:
    def __init__(self, cfg):
        self.timesteps = cfg.random_crop_kwargs.timesteps
        self.end_buffer = cfg.random_crop_kwargs.end_buffers

    def __call__(self, sample):
        x = sample
        crop_size = self.crop_size
        x = x[:, :, crop_size:-crop_size]
        return x

[(1998, 'event_1998_90', 'explosion', 64.209, 0),
 (1998, 'event_1998_91', 'earthquake', 4.777, 1),
 (1998, 'event_1998_145', 'explosion', 11.947, 2),
 (1998, 'event_1998_146', 'explosion', 14.849, 3),
 (1998, 'event_1998_195', 'explosion', 17.41, 4),
 (1998, 'event_1998_197', 'explosion', 19.352, 5),
 (1998, 'event_1998_201', 'explosion', 3.678, 6),
 (1998, 'event_1998_203', 'explosion', 9.447, 7),
 (1998, 'event_1998_206', 'earthquake', 4.936, 8),
 (1998, 'event_1998_216', 'earthquake', 7.525, 9),
 (1998, 'event_1998_234', 'explosion', 5.428, 10),
 (1998, 'event_1998_248', 'explosion', 2.053, 11),
 (1998, 'event_1998_274', 'explosion', 49.965, 12),
 (1998, 'event_1998_301', 'explosion', 20.586, 13),
 (1998, 'event_1998_308', 'explosion', 5.46, 14),
 (1998, 'event_1998_309', 'explosion', 4.536, 15),
 (1998, 'event_1998_311', 'explosion', 9.162, 16),
 (1998, 'event_1998_312', 'explosion', 6.873, 17),
 (1998, 'event_1998_314', 'explosion', 2.178, 18),
 (1998, 'event_1998_319', 'explosio

In [ ]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, events_list, label_dict, data_dict, transforms = None):
        """
        List(Tuple(int, str, str, float, int)): A list of tuples containing the year, event_id, label, snr and event_index.
        """
        self.events_list = events_list
        self.data_dict = data_dict
        self.label_dict = label_dict
        self.transforms = transforms
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    def __len__(self):
        """
        Returns:
            int: the size of the dataset
        """
        return len(self.events_list)

    def __getitem__(self, idx):
        year, event_id, label, snr, event_index = self.events_list[idx]
        sample = self.data_dict[year]["events"][event_id]
        if self.transforms:
            sample = self.transforms(sample)
        return sample, label, event_id